# Setting up experiment

# Scrap

In [1]:
import os
import sys
from tqdm import tqdm
import argparse
from pathlib import Path
from dataclasses import dataclass
from typing import Any, Dict, List,Tuple, Optional
import json
#
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.insert(0,module_path)
print('module_path : ', module_path)
from config import SciFactT5Config
import definitions
sys.path.append(os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))
print('definitions.PROJECT_VARS.ROOT_DIR : ', definitions.PROJECT_VARS.ROOT_DIR)
print('os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR) : ',os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR))
#

from T5ParEvo.src.data.data import Claim, Label, ClaimPredictions,GoldDataset
from T5ParEvo.src.linguistic.ner_abbr import NEREntity, Abbreviation
from multivers import util
from multivers.data_r import ClaimDataLoaderGenerator, get_dataloader, DataLoaderGenerator
from multivers.model_r import MultiVerSModel

from T5ParEvo.target_system.multivers.multivers_interface import PredictionParams, ModelPredictor

module_path :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
root dir :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
definitions.PROJECT_VARS.ROOT_DIR :  /home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo
os.path.dirname(definitions.PROJECT_VARS.ROOT_DIR) :  /home/qudratealahyratu/research/nlp/fact_checking/my_work
/home/qudratealahyratu/research/nlp/fact_checking/my_work/T5ParEvo


/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/pytorch_lightning/core/decorators.py:66: LightningDeprecationWarning: The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5. Please use `trainer.predict` instead for inference. The decorator was applied to `predict`
  "The `@auto_move_data` decorator is deprecated in v1.3 and will be removed in v1.5."


In [2]:
# This dataset to be used only for training
cfg= SciFactT5Config()
ds_train = GoldDataset(cfg.target_dataset.loc_target_dataset_corpus,
                    cfg.target_dataset.loc_target_dataset_train)
claim_train = ds_train.get_claim(39)

In [3]:

params = PredictionParams(
    checkpoint_path= "/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/checkpoints/scifact.ckpt",
    output_file= None,#"prediction/pred_opt_scifact.jsonl",
    batch_size=5,
    device=0,
    num_workers=4,
    no_nei=False,
    force_rationale=False,
    debug=False,
)
corpus_file = cfg.target_dataset.loc_target_dataset_corpus#cfg.target_dataset.loc_target_dataset_test#"/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/corpus.jsonl"
    

gold_claims = []
claims_path = cfg.target_dataset.loc_target_dataset_test#'/home/qudratealahyratu/research/nlp/fact_checking/my_work/multivers/data/scifact/claims_test_retrived.jsonl'
with open(claims_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        claim = Claim(id = data['id'], claim = data['claim'], cited_docs = data['doc_ids'], evidence = {},release = None)
        gold_claims.append(claim)

#get unique claims
unique_gold_claims = Claim.get_unique_claims(gold_claims)
# Predict for unique claims
# dataloader_generator = DataLoaderGenerator(params, unique_gold_claims, corpus_file)
# dataloader = dataloader_generator.get_dataloader_by_claims()
# predictor = ModelPredictor(params, dataloader)
# prediction_formatted = predictor.run()

In [4]:
unique_gold_claims[:10]

[Example 7: 10-20% of people with severe mental disorder receive no treatment in low and middle income countries.,
 Example 8: 25% of patients with melanoma and an objective response to PD-1 blockade will experience a progression in their melanoma.,
 Example 16: 50% of patients exposed to radiation have activated markers of myofibroblasts.,
 Example 23: 8% of burn patients are admitted for hospitalization and further treatment after appearing at hospital emergency wards or outpatient clinics.,
 Example 29: A breast cancer patient's capacity to metabolize tamoxifen has no effect on treatment outcome.,
 Example 31: A country's Vaccine Alliance (GAVI) eligibility is associated with accelerated adoption of the Hub vaccine.,
 Example 33: A deficiency of folate decreases blood levels of homocysteine.,
 Example 38: A deficiency of vitamin B6 increases blood levels of homocysteine.,
 Example 59: APOE4 expression in iPSC-derived neurons results in increased tau phosphorylation.,
 Example 65: AZ

In [5]:
from T5ParEvo.src.linguistic.ner_abbr import AbbreviationModel, NERModel, NEREntity, Abbreviation,ClaimProcessor
ner_models = [
    NERModel("en_ner_craft_md"),
    NERModel("en_ner_bc5cdr_md"),
    NERModel("en_ner_bionlp13cg_md"),
    NERModel("en_ner_jnlpba_md")
]

abbr_models = [
    # AbbreviationModel("en_core_sci_scibert"),
    # AbbreviationModel("en_core_sci_lg"),
    AbbreviationModel("en_ner_craft_md"),
    AbbreviationModel("en_ner_bc5cdr_md"),
    AbbreviationModel("en_ner_bionlp13cg_md"),
    AbbreviationModel("en_ner_jnlpba_md")
]

processor = ClaimProcessor(ner_models, abbr_models)
import spacy
spacy.prefer_gpu()
all_sentence_ner = []
all_sentence_abbr = []
for cur_claim in tqdm(unique_gold_claims[:]):
    ner_entities, abbreviations = processor.process_claim(cur_claim)
    all_sentence_ner.extend(ner_entities)
    all_sentence_abbr.extend(abbreviations)

/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_ner_craft_md' (0.5.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/qudratealahyratu/anaconda3/envs/scifact/lib/python3.7/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/qudratealahyratu/anaconda3/envs/scifact/lib/pytho

In [6]:
from collections import defaultdict

# Create a dictionary where each key is a claim ID and the value is a list of NEREntity objects for that claim
entities_by_claim = defaultdict(list)
for entity in all_sentence_ner:
    entities_by_claim[entity.claim_id].append(entity)

# Now you can merge the entities for each claim
merged_entities = {}
for claim_id, entities in entities_by_claim.items():
    merged_entities[claim_id] = NERModel.merge_overlapping_entities(entities)
    
# Create a dictionary where each key is a claim ID and the value is a list of Abbreviation objects for that claim
abbr_by_claim = defaultdict(list)
for abbr in all_sentence_abbr:
    abbr_by_claim[abbr.claim_id].append(abbr)

merged_abbreviations = {}
for claim_id, abbreviations in abbr_by_claim.items():
    merged_abbreviations[claim_id] = AbbreviationModel.merge_overlapping_abbreviations(abbreviations)    

In [14]:
#save the merged entities and abbreviations to ../data/meta/merged_entities.json and ../../data/meta/merged_abbreviations.json
import pickle

with open('../data/meta/merged_abbreviations.pkl', 'wb') as f:
    pickle.dump(merged_abbreviations, f)

with open('../data/meta/merged_entities.pkl', 'wb') as f:
    pickle.dump(merged_entities, f)
  

16

# Load Existing Dataframe to Prepare class

In [7]:
import pandas as pd

In [15]:
loc_df_scispacy_sentence_word_unq_ner_abr_filtered = '../data/meta/df_scispacy_sentence_word_unq_ner_abr_cust_2.pkl'
df_scispacy_sentence_word_unq_ner_abr_filtered = pd.read_pickle(loc_df_scispacy_sentence_word_unq_ner_abr_filtered)
df_scispacy_sentence_word_unq_ner_abr_filtered

,ner_text,ner_label,ner_model,claim,start_char,end_char,org_label,list_rationales,data_source,term_in_claim
0,UK,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,18,20,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,UK
1,PrP,GENE_OR_GENE_PRODUCT,en_ner_bionlp13cg_md,1 in 5 million in UK have abnormal PrP positiv...,35,38,REFUTES,"[RESULTS Of the 32,441 appendix samples 16 wer...",train,PrP
2,genomes,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",6,13,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,genomes
3,genetic sequence,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",41,57,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,genetic sequence
4,variants,SO,en_ner_craft_md,"1,000 genomes project enables mapping of genet...",139,147,SUPPORTS,"[In conclusion, uncommon or rare genetic varia...",dev,variants
...,...,...,...,...,...,...,...,...,...,...
1727,β-sheet,PROTEIN,en_ner_jnlpba_md,β-sheet opening occurs during pleurotolysin po...,0,7,SUPPORTS,[The major conformational changes in PlyB are ...,train,β-sheet
1728,pleurotolysin,PROTEIN,en_ner_jnlpba_md,β-sheet opening occurs during pleurotolysin po...,30,48,SUPPORTS,[The major conformational changes in PlyB are ...,train,pleurotolysin pore
1729,cRCT,EXPERIMENT,custom,Cost effectiveness evaluations based on cRCT d...,40,44,REFUTES,[CONCLUSIONS The published cost-effectiveness ...,train,cRCT
1730,embryonic stem cells (ESCs),TAXON,custom,Androgenetic haploid mouse embryonic stem cell...,27,54,SUPPORTS,[Our results demonstrate that AG-haESCs can be...,train,embryonic stem cells (ESCs)


In [13]:
"""    
    claim_id : int
    ner_text: str
    ner_label: str
    ner_model: str
    start_char: int
    end_char: int    
"""

for index, row in tqdm(df_scispacy_sentence_word_unq_ner_abr_filtered.iterrows()):
    # Access the 'sentence' and 'ner' columns of the row
    
    claim_txt = row['claim']
    ner_text = row['ner_text']
    ner_label = row['ner_label']
    ner_model = row['ner_model']
    start_char = row['start_char']
    end_char = row['end_char']
    term_in_claim = row['term_in_claim']
    # Do something with the data
    
    retrived_claim = Claim.get_claim_by_text(claims= unique_gold_claims, claim_text= claim_txt)
    if retrived_claim is None:
        print(f"Claim not found for claim_txt: {claim_txt}")
    else:
        entity = NEREntity(claim_id=retrived_claim.id, ner_text=ner_text, ner_label=ner_label, ner_model=ner_model, start_char=start_char, end_char=end_char)
        retrived_claim.ner_entities.append(entity)

101it [00:00, 2351.86it/s]

Claim not found for claim_txt: 1 in 5 million in UK have abnormal PrP positivity.
Claim not found for claim_txt: 1 in 5 million in UK have abnormal PrP positivity.
Claim not found for claim_txt: 1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.
Claim not found for claim_txt: 1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.
Claim not found for claim_txt: 1,000 genomes project enables mapping of genetic sequence variation consisting of rare variants with larger penetrance effects than common variants.
Claim not found for claim_txt: 1/2000 in UK have abnormal PrP positivity.
Claim not found for claim_txt: 1/2000 in UK have abnormal PrP positivity.
Claim not found for claim_txt: 32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.
Claim not found f

AttributeError: 'NoneType' object has no attribute 'append'

In [13]:
for cur_claim in tqdm(unique_gold_claims[:10]):
    print(cur_claim.claim)

100%|██████████| 10/10 [00:00<00:00, 15839.52it/s]

10-20% of people with severe mental disorder receive no treatment in low and middle income countries.
25% of patients with melanoma and an objective response to PD-1 blockade will experience a progression in their melanoma.
50% of patients exposed to radiation have activated markers of myofibroblasts.
8% of burn patients are admitted for hospitalization and further treatment after appearing at hospital emergency wards or outpatient clinics.
A breast cancer patient's capacity to metabolize tamoxifen has no effect on treatment outcome.
A country's Vaccine Alliance (GAVI) eligibility is associated with accelerated adoption of the Hub vaccine.
A deficiency of folate decreases blood levels of homocysteine.
A deficiency of vitamin B6 increases blood levels of homocysteine.
APOE4 expression in iPSC-derived neurons results in increased tau phosphorylation.
AZT given with ribavirin alleviates anemia.
